In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict, namedtuple, OrderedDict

In [2]:
rating_csv = 'dataset/ratings.csv'
movies_csv = 'dataset/movies.csv'

# Preprocessing

In [3]:
ratings_df = pd.read_csv(rating_csv)
ratings_df.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [4]:
movies_df = pd.read_csv(movies_csv)
movies_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


## Cleaning

In [5]:
ratings_df = ratings_df.drop(labels=['timestamp'], axis=1)

In [6]:
ratings_df.head(100)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
95,1,1445,3.0
96,1,1473,4.0
97,1,1500,4.0
98,1,1517,5.0


## Transactional dataset

In [7]:
ratings_df = ratings_df[ratings_df['rating'] > 2]

In [8]:
ratings_df.head(100)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
96,1,1473,4.0
97,1,1500,4.0
98,1,1517,5.0
99,1,1552,4.0


In [9]:
user_counts = pd.DataFrame(ratings_df['userId'].value_counts().sort_values())
user_counts

,count
userId,
442,2
508,6
293,10
329,11
431,13
...,...
610,1233
448,1255
599,1794


In [10]:
active_users = user_counts[user_counts['count'] > 10].index
ratings_df = ratings_df[ratings_df['userId'].isin(active_users)]

In [11]:
ratings_df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [12]:
transactions = (ratings_df
                 .groupby('userId')['movieId']
                 .apply(lambda x: (list(set(x)), len(x)))
                 .reset_index())

transactions[['movies', 'count']] = pd.DataFrame(transactions['movieId'].tolist(), index=transactions.index)
transactions.drop(columns=['movieId'], inplace=True)

transactions

,userId,movies,count
0,1,"[1024, 1, 1025, 3, 2048, 1029, 6, 1030, 1031, ...",226
1,2,"[115713, 122882, 48516, 91529, 80906, 91658, 1...",28
2,3,"[70946, 2851, 5764, 4518, 26409, 7991, 1275, 2...",18
3,4,"[1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ...",167
4,5,"[1, 515, 261, 265, 527, 531, 21, 150, 534, 153...",40
...,...,...,...
602,606,"[1, 8195, 6148, 7, 11, 69644, 4109, 15, 17, 18...",1070
603,607,"[1, 517, 2053, 2054, 1544, 3081, 11, 1036, 257...",174
604,608,"[1, 4105, 10, 6157, 16, 21, 31, 32, 2080, 34, ...",670
605,609,"[1, 137, 10, 650, 1161, 786, 150, 288, 161, 10...",37


In [13]:
transactions.to_csv('transaction_dataset.csv', header=True)

In [14]:
transactions = pd.read_csv('transaction_dataset.csv')
transactions.head(10)

,Unnamed: 0,userId,movies,count
0,0,1,"[1024, 1, 1025, 3, 2048, 1029, 6, 1030, 1031, ...",226
1,1,2,"[115713, 122882, 48516, 91529, 80906, 91658, 1...",28
2,2,3,"[70946, 2851, 5764, 4518, 26409, 7991, 1275, 2...",18
3,3,4,"[1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ...",167
4,4,5,"[1, 515, 261, 265, 527, 531, 21, 150, 534, 153...",40
5,5,6,"[2, 3, 4, 5, 6, 7, 8, 515, 10, 11, 516, 13, 52...",294
6,6,7,"[1, 8207, 34319, 42002, 3114, 1584, 50, 58, 16...",111
7,7,8,"[2, 11, 141, 527, 21, 150, 282, 539, 32, 34, 2...",43
8,8,9,"[3328, 4993, 5378, 5890, 5893, 1674, 5902, 373...",34
9,9,10,"[7169, 33794, 6155, 54286, 72720, 86548, 30749...",119


In [15]:
def split_movies(movies, test_size=0.2):
    movies = json.loads(movies)  # Convert set to list
    split_idx = int(len(movies) * (1 - test_size))  # Index for 80/20 split
    train_movies = set(movies[:split_idx])  # First 80% for training
    test_movies = set(movies[split_idx:])  # Last 20% for test
    return (train_movies, test_movies)

transactions[['trainMovies', 'testMovies']] = transactions['movies'].apply(
    lambda movies: pd.Series(split_movies(movies))
)
transactions = transactions.drop(labels='movies', axis=1)

In [16]:
transactions.head(10)

,Unnamed: 0,userId,count,trainMovies,testMovies
0,0,1,226,"{1024, 1, 1025, 3, 2048, 1029, 6, 1030, 1031, ...","{2459, 3489, 1954, 1445, 2470, 423, 4006, 2985..."
1,1,2,28,"{115713, 122882, 48516, 91529, 80906, 91658, 1...","{46970, 80489, 71535, 74458, 6874, 8798}"
2,2,3,18,"{70946, 2851, 5764, 4518, 3703, 26409, 2288, 8...","{1371, 5181, 7899, 5919}"
3,3,4,167,"{1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ...","{904, 908, 910, 912, 914, 919, 920, 1947, 3996..."
4,4,5,40,"{1, 515, 261, 265, 527, 531, 21, 150, 534, 153...","{608, 232, 364, 110, 367, 247, 253, 349}"
5,5,6,294,"{2, 3, 4, 5, 6, 7, 8, 515, 10, 11, 516, 13, 52...","{509, 510, 405, 410, 412, 415, 416, 505, 419, ..."
6,6,7,111,"{1, 34319, 8207, 42002, 3114, 1584, 50, 58, 16...","{4995, 3977, 33162, 6539, 920, 3994, 924, 1954..."
7,7,8,43,"{2, 11, 141, 527, 21, 150, 282, 539, 32, 34, 2...","{235, 364, 236, 110, 367, 252, 377, 380, 253}"
8,8,9,34,"{3328, 4993, 5378, 5890, 5893, 1674, 5902, 373...","{5481, 5872, 6001, 371, 627, 1270, 2300}"
9,9,10,119,"{7169, 33794, 6155, 54286, 72720, 86548, 30749...","{103335, 103339, 78772, 81845, 104374, 81847, ..."


In [17]:
inp = transactions[['userId', 'trainMovies']]
inp.columns = ['Id', 'tr']
n_inp = inp.shape[0]

inp


,Id,tr
0,1,"{1024, 1, 1025, 3, 2048, 1029, 6, 1030, 1031, ..."
1,2,"{115713, 122882, 48516, 91529, 80906, 91658, 1..."
2,3,"{70946, 2851, 5764, 4518, 3703, 26409, 2288, 8..."
3,4,"{1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ..."
4,5,"{1, 515, 261, 265, 527, 531, 21, 150, 534, 153..."
...,...,...
602,606,"{1, 8195, 6148, 7, 11, 69644, 4109, 15, 17, 18..."
603,607,"{1, 2053, 517, 2054, 1544, 3081, 11, 1036, 257..."
604,608,"{1, 4105, 10, 6157, 16, 21, 31, 32, 2080, 34, ..."
605,609,"{1, 137, 10, 1161, 650, 786, 150, 288, 161, 10..."


In [18]:
min_sup = 50
min_conf = 0.1

## FPTree

### Construct FPTree

In [19]:
class FPNode:
    def __init__(self, item, count, parent) -> None:
        self.item = item
        self.parent = parent
        self.children = {}
        self.link = None
        self.count = count

    def increment(self, count):
        self.count = self.count + count

    def pretty_print(self):
        print(f'({self.item}-{self.count})[ ', end='')
        for id, ch in self.children.items():
            ch.pretty_print()
        print(' ], ', end='')


class FPTreeBase:
    def __init__(self, transactions, freq_1_itemsets, min_sup) -> None:
        self.transactions = transactions
        self.freq_1_itemsets = freq_1_itemsets
        self.min_sup = min_sup
        self.header_table = defaultdict(list)
        self.root = None

    def _build_tree(self):
        root = FPNode(None, 1, None)
        for transaction in self.transactions:
            self._insert_transaction(root, transaction)
        self.root = root
        return root

    def _insert_transaction(self, node, transaction):
        if len(transaction) == 0:
            return

        [first_item, freq] = transaction[0]
        if first_item in node.children:
            node.children[first_item].increment(freq)
        else:
            new_node = FPNode(first_item, freq, node)
            node.children[first_item] = new_node
            self.header_table[first_item].append(new_node)

        # Recursively insert the rest of the transaction
        remaining_tr = transaction[1:]
        self._insert_transaction(node.children[first_item], remaining_tr)

    def _prefix_path(self, node):
        path = []
        while node and node.item is not None:
            path.append(node.item)
            node = node.parent
        return path[::-1]

    def mine_patterns(self, depth=0):
        patterns = {}
        final_cond_base = []
        # Process items in reverse order of their frequency (from the header table)
        for item in reversed(self.freq_1_itemsets):
            if (item not in self.header_table):
                continue

            conditional_patterns = []
            for node in self.header_table[item]:
                path = self._prefix_path(node)
                # print('Path for ', item, ':', path[1:10],'...')
                if path:
                    conditional_patterns.append(
                        [[p, node.count] for p in path])

            # print(f'Conditional Base Patterns I{item}:', conditional_patterns)
            leaf_item_freq = OrderedDict()
            for pattern in conditional_patterns:
                for [node, freq] in pattern:
                    if node not in leaf_item_freq:
                        leaf_item_freq[node] = freq
                    else:
                        leaf_item_freq[node] += freq

            # print(leaf_item_freq)
            leaf_item_freq = {k: v for k,
                              v in leaf_item_freq.items() if v >= self.min_sup}
            # print(leaf_item_freq)

            filtered_patterns = []
            for pattern in conditional_patterns:
                fil = list(filter(lambda p: p[0] in leaf_item_freq, pattern))
                filtered_patterns.append(fil)
            conditional_patterns = filtered_patterns

            for pattern in conditional_patterns:
                temp = []
                for [node, freq] in pattern:
                    if (node in leaf_item_freq):
                        temp.append([node, freq])

                final_cond_base.append(temp)

            # print(f'Final Conditional Base Patterns I{item}:', conditional_patterns)
            if (len(conditional_patterns) == 1):
                if (conditional_patterns[0][0][1] >= self.min_sup):
                    patterns[tuple(sorted(
                        [node for node, freq in conditional_patterns[0]]))] = conditional_patterns[0][0][1]
            elif (len(final_cond_base) >= 2):
                print('Recursing >>>>>>>>', depth+1)
                conditional_tree = FPTreeBase(
                    [p[0:len(p)-1] for p in conditional_patterns], self.freq_1_itemsets, self.min_sup)
                conditional_tree._build_tree()

            #   conditional_tree.root.pretty_print()
            #   print()

                conditional_patterns_freq = conditional_tree.mine_patterns(
                    depth=depth+1)
                for pattern, count in conditional_patterns_freq.items():
                    patterns[tuple(sorted(list(pattern) + [item]))] = count
                print('BackTracking <<<<<<<<')
            #   print(f'Patterns ending in I{item}:', conditional_patterns_freq)

            item_count = sum([pattern[0][1]
                             for pattern in conditional_patterns])
            if item_count >= self.min_sup:
                patterns[(item,)] = item_count

        # unique_cond_base_set = set(map(tuple,final_cond_base))
        # unique_cond_base_list =list(unique_cond_base_set)

        return patterns


class FPTree(FPTreeBase):
    def __init__(self, transactions, min_sup) -> None:
        freq_1_itemsets, freq_1_df = self.__generate_freq_1_itemsets(transactions)
        self.freq_1_itemsets = freq_1_itemsets
        self.freq_1_df = freq_1_df
        transactions = self.__filter_sort_transactions(
            transactions, freq_1_itemsets)

        new_transactions = []
        for transaction in transactions:
            new_transactions.append([[node, 1] for node in transaction])

        super().__init__(new_transactions, freq_1_itemsets, min_sup)

    def __generate_freq_1_itemsets(self, transactions):
        it_set_1 = set()
        for tr in transactions:
            it_set_1 = it_set_1.union(set(tr))
        it_set_1 = list(it_set_1)
        print('distinct:', len(it_set_1))

        mp = {}
        for tr in transactions:
            for mId in tr:
                if mId in mp:
                    mp[mId] += 1
                else:
                    mp[mId] = 1

        it_df_1 = pd.DataFrame(list(mp.items()), columns=['mId', 'Support'])
        it_df_1 = it_df_1.sort_values(by='Support', ascending=False)
        it_df_1 = it_df_1[it_df_1['Support'] >= min_sup]
        print('distinct movies after filtering minsup:', it_df_1.shape[0])

        it_set_1 = it_df_1['mId'].tolist()
        return it_set_1, it_df_1

    def __filter_sort_transactions(self, transactions, freq_1_itemsets):
        def fn(movies):
            movies = list(movies)
            filtered_movies = filter(lambda x: (x in freq_1_itemsets), movies)
            sorted_movies = sorted(
                filtered_movies, key=lambda x: freq_1_itemsets.index(x))
            return sorted_movies

        transactions = [fn(tr) for tr in transactions]
        return transactions

    def fit(self):
        self.root = self._build_tree()
        return self.freq_1_df
    
    def mine_patterns(self):
        patterns = super().mine_patterns()
        return patterns, self.freq_1_df

In [20]:
fpTree = FPTree(inp['tr'].tolist(), min_sup)
fpTree.fit()
print('FItting done')
patterns, freq_1_df = fpTree.mine_patterns()
print('Found', len(patterns), 'Frequent itemsets')

patterns = pd.DataFrame(list(patterns.items()), columns=['itemset','Support'])
patterns = patterns.sort_values(by='Support', ascending=False)
patterns = patterns.reset_index(drop=True)
patterns

distinct: 7762
distinct movies after filtering minsup: 264
FItting done
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
BackTracking <<<<<<<<
Recursing >>>>>>>> 1
Back

,itemset,Support
0,"(318,)",307
1,"(296,)",289
2,"(593,)",265
3,"(2571,)",264
4,"(356,)",240
...,...,...
68332,"(260, 296, 858, 1198, 2762)",50
68333,"(260, 589, 858, 1198, 2762)",50
68334,"(858, 1196, 1198, 2571, 2762)",50
68335,"(260, 858, 1196, 1198, 2571, 2762)",50


In [21]:
patterns.to_csv('frequent-itemsets.csv',index=False,header=True, encoding='utf-8')
freq_1_df.to_csv('1-itemset-frequency.csv', index=False, header=True, encoding='utf-8')

In [38]:
import ast

freq_1_df = pd.read_csv('1-itemset-frequency.csv')
patterns = pd.read_csv('frequent-itemsets.csv')
patterns['movies'] = patterns['itemset'].apply(lambda x: set(ast.literal_eval(x)))
patterns.drop(columns='itemset', axis=1, inplace=True)
patterns

,Support,movies
0,307,{318}
1,289,{296}
2,265,{593}
3,264,{2571}
4,240,{356}
...,...,...
68332,50,"{260, 296, 2762, 1198, 858}"
68333,50,"{260, 2762, 589, 1198, 858}"
68334,50,"{2762, 2571, 1196, 1198, 858}"
68335,50,"{260, 2762, 2571, 1196, 1198, 858}"


In [39]:
association_df = []

for id, row in patterns.iterrows():
  support, movies = row
  if(len(movies) > 1):
    for mv in movies:
      X = mv
      Y = movies.copy()
      Y.discard(X)
      association_df.append([X, Y, support])

association_df = pd.DataFrame(association_df, columns=['X', 'Y', 'Support'])
association_df = association_df.sort_values(by=['Support'], ascending=False)

freq_1_df['X Freq'] = freq_1_df['Support']
freq_1_df['X'] = freq_1_df['mId']
freq_1_df.drop(columns=['mId', 'Support'], axis=1,inplace=True)

association_df = pd.merge(association_df, freq_1_df, on='X', how='left')
association_df['Confidence'] = association_df['Support']/association_df['X Freq']
association_df = association_df[association_df['Confidence'] >= min_conf]

association_df['Y_formatted'] = association_df['Y'].apply(lambda y: str(y))
association_df['Rule'] = association_df['X'].astype(str) + ' -> ' + association_df['Y_formatted']

association_df = association_df.drop(columns=['Y_formatted'], axis=1)

association_df

,X,Y,Support,X Freq,Confidence,Rule
0,318,{296},209,307,0.680782,318 -> {296}
1,296,{318},209,289,0.723183,296 -> {318}
2,296,{593},194,289,0.671280,296 -> {593}
3,593,{296},194,265,0.732075,593 -> {296}
4,318,{593},189,307,0.615635,318 -> {593}
...,...,...,...,...,...,...
296937,1197,"{260, 1210, 2858, 2571, 1196}",50,139,0.359712,"1197 -> {260, 1210, 2858, 2571, 1196}"
296938,1210,"{260, 2858, 2571, 1196, 1197}",50,186,0.268817,"1210 -> {260, 2858, 2571, 1196, 1197}"
296939,593,"{2571, 1213, 1214}",50,265,0.188679,"593 -> {2571, 1213, 1214}"
296940,2571,"{593, 1213, 1214}",50,264,0.189394,"2571 -> {593, 1213, 1214}"


In [41]:
association_df.to_csv('association_rules.csv', index=False,header=True, encoding='utf-8')

In [42]:
support_sorted_df = association_df.sort_values(by='Support', ascending=False).reset_index(drop=True)
support_sorted_df = support_sorted_df.head(100)
support_sorted_df

,X,Y,Support,X Freq,Confidence,Rule
0,296,{318},209,289,0.723183,296 -> {318}
1,318,{296},209,307,0.680782,318 -> {296}
2,296,{593},194,289,0.671280,296 -> {593}
3,593,{296},194,265,0.732075,593 -> {296}
4,593,{318},189,265,0.713208,593 -> {318}
...,...,...,...,...,...,...
95,1210,"{2571, 260}",131,186,0.704301,"1210 -> {2571, 260}"
96,2571,"{1210, 260}",131,264,0.496212,"2571 -> {1210, 260}"
97,296,{1196},131,289,0.453287,296 -> {1196}
98,260,"{1210, 2571}",131,239,0.548117,"260 -> {1210, 2571}"


In [46]:
support_sorted_df.to_csv('support_sorted_top_100.csv', index=False,header=True, encoding='utf-8')

top_100_support_list = support_sorted_df['Rule'].tolist()
with open('3_top100RulesBySup.txt', 'w+') as f:
  f.write('\n'.join(top_100_support_list))

In [44]:
confidence_sorted_df = association_df.sort_values(by='Confidence', ascending=False).reset_index(drop=True)
confidence_sorted_df = confidence_sorted_df.head(100)
confidence_sorted_df

,X,Y,Support,X Freq,Confidence,Rule
0,33493,{2571},61,63,0.968254,33493 -> {2571}
1,33493,{260},61,63,0.968254,33493 -> {260}
2,33493,{1196},61,63,0.968254,33493 -> {1196}
3,6934,{2571},55,57,0.964912,6934 -> {2571}
4,33493,"{2571, 1196}",60,63,0.952381,"33493 -> {2571, 1196}"
...,...,...,...,...,...,...
95,1258,{296},79,94,0.840426,1258 -> {296}
96,4973,{356},68,81,0.839506,4973 -> {356}
97,2683,{260},68,81,0.839506,2683 -> {260}
98,1291,{1196},114,136,0.838235,1291 -> {1196}


In [47]:
confidence_sorted_df.to_csv('confidence_sorted_top_100.csv', index=False,header=True, encoding='utf-8')

top_100_support_list = support_sorted_df['Rule'].tolist()
with open('3_top100RulesByConf.txt', 'w+') as f:
  f.write('\n'.join(top_100_support_list))

In [48]:
support_sorted_df['Y'] = support_sorted_df['Y'].apply(lambda x: tuple(sorted(list(x))))
confidence_sorted_df['Y'] = confidence_sorted_df['Y'].apply(lambda x: tuple(sorted(list(x))))

In [50]:
common_df = pd.merge(support_sorted_df, confidence_sorted_df, how='inner')
common_df = common_df.sort_values(by='Confidence', ascending=False).reset_index(drop=True)
common_df

,X,Y,Support,X Freq,Confidence,Rule
0,1196,"(260,)",182,204,0.892157,1196 -> {260}
1,1210,"(260,)",165,186,0.887097,1210 -> {260}
